# Stoxly - Finansal Analiz Örneği

Bu notebook, Stoxly'nin finansal analiz modüllerini kullanarak örnek analizler yapar.


In [ ]:
# Gerekli kütüphaneleri import et
import sys
sys.path.append('..')

from lib.mock_service import fetch_data, generate_mock_data
from lib.financial_analysis import calculate_all_indicators
from lib.text_translator import translate_indicators, get_risk_level
from lib.types import PriceData
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


## 1. Veri Çekme


In [ ]:
# Örnek: GARAN hissesi için veri çek
ticker = "GARAN"
price_data = fetch_data(ticker, "stock")

print(f"Toplam {len(price_data)} günlük veri çekildi.")
print(f"İlk tarih: {price_data[0].date}")
print(f"Son tarih: {price_data[-1].date}")
print(f"Son fiyat: {price_data[-1].close} TL")


## 2. Teknik Göstergeleri Hesapla


In [ ]:
# Tüm teknik göstergeleri hesapla
indicators = calculate_all_indicators(price_data)

print("=== Teknik Göstergeler ===")
print(f"RSI: {indicators.rsi:.2f}")
print(f"SMA 20: {indicators.sma20:.2f} TL")
print(f"SMA 50: {indicators.sma50:.2f} TL")
print(f"SMA 200: {indicators.sma200:.2f} TL")
print(f"Volatilite: %{indicators.volatility:.2f}")
print(f"Risk Skoru: {indicators.risk_score:.1f}/100")
print(f"Güncel Fiyat: {indicators.current_price:.2f} TL")


## 3. Türkçe Mesajlara Çevir


In [ ]:
# Göstergeleri Türkçe mesajlara çevir
insights = translate_indicators(indicators)
risk_level = get_risk_level(indicators.risk_score)

print("=== Analiz Sonuçları ===")
print(f"\n📊 RSI Analizi:")
print(f"   {insights.rsi_message}")
print(f"\n⚖️ Risk Analizi:")
print(f"   Risk Seviyesi: {risk_level}")
print(f"   {insights.risk_message}")
print(f"\n📈 Trend Analizi:")
print(f"   {insights.trend_message}")

if insights.main_warning:
    print(f"\n⚠️ Ana Uyarı: {insights.main_warning}")

if insights.main_action:
    print(f"\n💡 Öneri: {insights.main_action}")


## 4. Görselleştirme


In [ ]:
# DataFrame'e çevir
df = pd.DataFrame([
    {
        'date': p.date,
        'open': p.open,
        'high': p.high,
        'low': p.low,
        'close': p.close,
        'volume': p.volume
    }
    for p in price_data
])

df.set_index('date', inplace=True)

# Grafik oluştur
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Fiyat grafiği
axes[0].plot(df.index, df['close'], label='Kapanış Fiyatı', linewidth=2)
axes[0].axhline(y=indicators.sma20, color='blue', linestyle='--', label=f'SMA 20 ({indicators.sma20:.2f})', alpha=0.7)
axes[0].axhline(y=indicators.sma50, color='orange', linestyle='--', label=f'SMA 50 ({indicators.sma50:.2f})', alpha=0.7)
axes[0].axhline(y=indicators.sma200, color='red', linestyle='--', label=f'SMA 200 ({indicators.sma200:.2f})', alpha=0.7)
axes[0].set_ylabel('Fiyat (TL)', fontsize=12)
axes[0].set_title(f'{ticker} - Fiyat Hareketi ve Moving Averages', fontsize=14, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Hacim grafiği
axes[1].bar(df.index, df['volume'], alpha=0.6, color='lightblue')
axes[1].set_ylabel('Hacim', fontsize=12)
axes[1].set_xlabel('Tarih', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. Karşılaştırmalı Analiz


In [ ]:
# Birden fazla hisseyi karşılaştır
tickers = ["GARAN", "AKBNK", "THYAO"]
results = []

for ticker in tickers:
    try:
        data = fetch_data(ticker, "stock")
        ind = calculate_all_indicators(data)
        risk = get_risk_level(ind.risk_score)
        
        results.append({
            'Ticker': ticker,
            'Fiyat': ind.current_price,
            'RSI': ind.rsi,
            'Risk Skoru': ind.risk_score,
            'Risk Seviyesi': risk,
            'Volatilite': ind.volatility
        })
    except Exception as e:
        print(f"{ticker} için hata: {e}")

comparison_df = pd.DataFrame(results)
print("\n=== Karşılaştırmalı Analiz ===")
print(comparison_df.to_string(index=False))
